In [ ]:
import torch
import numpy as np

## Interpolation in Latent Space

Since the autoencoder help to transform the space into more representive, less feature complexity, and clustering the paretor solutions depending on similarity. We can utilize it to query new pareto solution for arabitrary defined distance by the user (7.5km)
Since the optimum pareto solution could not be the best choice we can interpolate it for better result


### Interploation tactics:
- Linear Interploation:
    1. search for the optimum solution in the closest cluster, e.g. if the request is something like (14Km, 0.8 time, 0.2 energy), then the autoencoder should know to narrow down the search for better pareto front accessibility.

    2. apply linear interpolation, taking the one of the optimal point in the pareto front. e.g if the time is more improtant then we have to shift the searching to area wher the time is more important and then select the opmtimum point, then we apply the interpolation on it.

    3. decoded the point to get the corrosponding $u_t$ input

    4. check the feasibilty and validity of the generated new input and deploy to MonoCap
    
- Linear Interpolation
For solutions $\mathbf{x}_A$, $\mathbf{x}_B$:
    1. Encode two solutions:
    $$
    \mathbf{z}_A = \text{Encoder}(\mathbf{x}_A), \quad \mathbf{z}_B = \text{Encoder}(\mathbf{x}_B)
    $$

    2. Linear interpolation:
    $$
    \mathbf{z}_{new} = \alpha\mathbf{z}_A + (1-\alpha)\mathbf{z}_B, \quad \alpha \in [0,1]
    $$

    3. Decode to generate new solution:
    $$
    \mathbf{x}_{new} = \text{Decoder}(\mathbf{z}_{new})
    $$


- Geodesic Interpolation
    For solutions $\mathbf{x}_A$, $\mathbf{x}_B$:
    1. Encode: $\mathbf{z}_A = g_\phi(\mathbf{x}_A)$, $\mathbf{z}_B = g_\phi(\mathbf{x}_B)$
    2. Spherical interpolation:
    $$
    \mathbf{z}_{\text{new}} = \frac{\sin[(1-\alpha)\Omega]}{\sin\Omega}\mathbf{z}_A + \frac{\sin[\alpha\Omega]}{\sin\Omega}\mathbf{z}_B
    $$
    where $\Omega = \arccos(\mathbf{z}_A^\top \mathbf{z}_B)$

    3. Decode:
    $$
    \mathbf{x}_{\text{new}} = f_\theta(\mathbf{z}_{\text{new}})
    $$


In [ ]:
def barycentric(z_list, d_list, tau=1.0):
    # d_list: distances in condition space
    w = np.exp(-(np.array(d_list) ** 2) / tau)
    w = w / w.sum()
    z_arr = np.stack(z_list)
    return (w[:, None] * z_arr).sum(axis=0)


# Interpolation with geodesic sampling
def geodesic_interpolate(z1, z2, alpha):
    omega = np.arccos(np.clip(np.dot(z1.T, z2)[0, 0], -1, 1))
    return (np.sin((1 - alpha) * omega) / np.sin(omega)) * z1 + (
        np.sin(alpha * omega) / np.sin(omega)
    ) * z2


z_A = encoder.predict(X_train[0:1])
z_B = encoder.predict(X_train[1:2])

for alpha in np.linspace(0, 1, 5):
    z_new = geodesic_interpolate(z_A, z_B, alpha)
    J_new = decoder.predict(z_new)
    J_original = scaler.inverse_transform(J_new)
    print(
        f"α={alpha:.1f}: Time={J_original[0,0]:.2f}s, Energy={J_original[0,1]:.2f}kWh"
    )


In [ ]:
# Interpolation with linear sampling

# Encode two Pareto solutions
z_A = encoder.predict(X_train[0:1])  # Solution A
z_B = encoder.predict(X_train[1:2])  # Solution B

# Linear interpolation
alpha = 0.5
z_new = alpha * z_A + (1 - alpha) * z_B

# Decode to generate new solution
J_new = decoder.predict(z_new)

# Denormalize
J_new_original = scaler.inverse_transform(J_new)
print(
    f"Interpolated Solution: Time = {J_new_original[0, 0]:.2f} s, Energy = {J_new_original[0, 1]:.2f} kWh"
)

In [ ]:
class MonocapController:
    def __init__(self, cvae, prior, index, physics_model):
        self.cvae = cvae.eval()
        self.prior = prior.eval()
        self.index = index
        self.physics = physics_model

    def query(self, c_new):
        # Option A: use prior
        mu_star = self.prior(torch.tensor(c_new).float().to(self.cvae.device))
        z_star = mu_star.cpu().numpy()
        # Option B: use KD-tree interpolation
        anchors = self.index.query(c_new, k=3)
        z_list, d_list = zip(*anchors)
        z_star = barycentric(z_list, d_list)

        # Decode
        z_t = torch.tensor(z_star).unsqueeze(0).to(self.cvae.device)
        c_t = torch.tensor(c_new, dtype=torch.float32).unsqueeze(0).to(self.cvae.device)
        with torch.no_grad():
            profile, _, _ = self.cvae(z_t, c_t)
        # Validate
        t, e, mj, xj = self.physics(profile, c_t)
        if e.item() > c_new[1] or xj.item() > self.cvae.max_jerk:
            # fallback or refine
            pass
        return profile.cpu().numpy()


In [ ]:
controller = MonocapController(
    cvae=monocap_cvae,
    prior=monocap_prior,
    index=monocap_index,
    physics_model=monocap_physics_model,
)

# Get minimum time solution for 10km
fast_profile = controller.query(
    distance_km=10, time_preference=0.9, energy_preference=0.1
)

# Get balanced solution for 15km
balanced_profile = controller.query(
    distance_km=15, time_preference=0.5, energy_preference=0.5
)

# Get energy-efficient solution for 20km
efficient_profile = controller.query(
    distance_km=20, time_preference=0.2, energy_preference=0.8
)

## Solution Validation Protocol

### Dominance Verification
$$
\mathbf{x}_{\text{new}} \text{ is non-dominated iff } \nexists \mathbf{x}_i \in X :
\begin{cases}
f_1^{(i)} \leq f_1^{\text{(new)}} \\
f_2^{(i)} \leq f_2^{\text{(new)}} \\
\|\mathbf{x}_i - \mathbf{x}_{\text{new}}\|_2 > \delta
\end{cases}
$$

### Feasibility Check
$$
\mathbf{x}_{\text{new}} \in \mathcal{F} \iff
\begin{cases}
g_1(\mathbf{x}_{\text{new}}) \leq 0 \\
g_2(\mathbf{x}_{\text{new}}) \leq 0 \\
\vdots \\
g_k(\mathbf{x}_{\text{new}}) \leq 0
\end{cases}
$$

Ensure:
$$
f_1^{new} \geq 0, \quad f_2^{new} \geq 0
$$
And any problem-specific constraints (e.g., $g(\mathbf{x}_{new}) \leq 0$)

### Check mapping back to decision space (acceleratio, decelereation)